In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("titanic.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'],axis='columns',inplace=True)

In [4]:
df.head()

,Survived,Pclass,Sex,Age,Fare
0,0,3,male,22.0,7.2500
1,1,1,female,38.0,71.2833
2,1,3,female,26.0,7.9250
3,1,1,female,35.0,53.1000
4,0,3,male,35.0,8.0500


In [5]:
inputs = df.drop('Survived',axis='columns')
target = df.Survived

In [9]:
inputs.isnull().sum()

Pclass      0
Sex         0
Age       177
Fare        0
dtype: int64

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [18]:
# Define the imputer for 'age' column (mean imputation)
age_imputer = SimpleImputer(strategy='mean')

# Define the one-hot encoder for the 'sex' column
sex_encoder = OneHotEncoder(drop='first')

# Create the column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', age_imputer, ['Age']),
        ('cat', sex_encoder, ['Sex'])
    ],
    remainder='passthrough'  # Keep other columns unchanged
)

In [19]:
transformed_data = preprocessor.fit_transform(inputs)

In [20]:
new_columns = ['Age', 'Sex_male', 'Pclass', 'Fare']  # Explicit names based on transformations and passthrough

# Convert the numpy array back to a DataFrame
transformed_df = pd.DataFrame(transformed_data, columns=new_columns)

In [21]:
transformed_df

,Age,Sex_male,Pclass,Fare
0,22.000000,1.0,3.0,7.2500
1,38.000000,0.0,1.0,71.2833
2,26.000000,0.0,3.0,7.9250
3,35.000000,0.0,1.0,53.1000
4,35.000000,1.0,3.0,8.0500
...,...,...,...,...
886,27.000000,1.0,2.0,13.0000
887,19.000000,0.0,1.0,30.0000
888,29.699118,0.0,3.0,23.4500
889,26.000000,1.0,1.0,30.0000


In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(transformed_df,target,test_size=0.2)

In [24]:
len(X_train)

712

In [25]:
from sklearn import tree
model = tree.DecisionTreeClassifier()

In [26]:
model.fit(X_train,y_train)

DecisionTreeClassifier()

In [27]:
model.score(X_test,y_test)

0.7486033519553073

In [28]:
y_pred=model.predict(X_test)

In [31]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.77      0.83      0.80       108
           1       0.71      0.62      0.66        71

    accuracy                           0.75       179
   macro avg       0.74      0.73      0.73       179
weighted avg       0.75      0.75      0.75       179

